In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *


from preprocess import water_process
# from model.cnn import cnn_4layer, mlp_layer
from model.mlp import mlp_layer

2024-01-30 16:44:32.210375: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
np.percentile(data['MEAN_RAIN'], 95)

0.0053125

### Preprocess

In [5]:
# ====== preprocessing parameters ======
n_hours = 72
k = 24 
masked_value = 1e-10
split_1 = 0.7
split_2 = 0.85

In [6]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = water_process(n_hours, k, masked_value, split_1, split_2)

Index(['MEAN_RAIN', 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2',
       'GATE_S26_1', 'GATE_S26_2', 'PUMP_S25B', 'PUMP_S26', 'HWS_S25A',
       'HWS_S25B', 'HWS_S26', 'WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26'],
      dtype='object')


In [7]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(67435, 96, 16) (14451, 96, 16) (14451, 96, 16) (67435, 96) (14451, 96) (14451, 96)


In [8]:
data = pd.read_csv("../../data/Merged-update_hourly.csv", index_col=0)
data.fillna(0, inplace=True)
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


### Model

In [13]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k*4
input_shape = train_X_mask.shape[1:]

In [14]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [15]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('../../saved_models/water_all_95_cp.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True
                    )


model.fit([train_X_mask], train_ws_y,
          validation_data=([val_X_mask], val_ws_y),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

Epoch 1/1000
132/132 - 6s - loss: 0.0423 - mae: 0.1603 - val_loss: 0.0242 - val_mae: 0.1312

Epoch 00001: val_mae improved from inf to 0.13124, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 2/1000
132/132 - 3s - loss: 0.0090 - mae: 0.0774 - val_loss: 0.0058 - val_mae: 0.0601

Epoch 00002: val_mae improved from 0.13124 to 0.06008, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 3/1000
132/132 - 3s - loss: 0.0017 - mae: 0.0316 - val_loss: 0.0031 - val_mae: 0.0445

Epoch 00003: val_mae improved from 0.06008 to 0.04454, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 4/1000
132/132 - 3s - loss: 0.0012 - mae: 0.0263 - val_loss: 0.0025 - val_mae: 0.0401

Epoch 00004: val_mae improved from 0.04454 to 0.04014, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 5/1000
132/132 - 3s - loss: 8.7276e-04 - mae: 0.0218 - val_loss: 0.0023 - val_mae: 0.0382

Epoch 00005: val_mae improved from 0.04014 to 0.03819, saving model to ../../saved_models/water_all


Epoch 00043: val_mae improved from 0.02442 to 0.02411, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 44/1000
132/132 - 3s - loss: 1.7361e-04 - mae: 0.0082 - val_loss: 9.0587e-04 - val_mae: 0.0247

Epoch 00044: val_mae did not improve from 0.02411
Epoch 45/1000
132/132 - 3s - loss: 1.7139e-04 - mae: 0.0082 - val_loss: 8.7090e-04 - val_mae: 0.0242

Epoch 00045: val_mae did not improve from 0.02411
Epoch 46/1000
132/132 - 3s - loss: 1.7002e-04 - mae: 0.0081 - val_loss: 8.7517e-04 - val_mae: 0.0243

Epoch 00046: val_mae did not improve from 0.02411
Epoch 47/1000
132/132 - 3s - loss: 1.6859e-04 - mae: 0.0081 - val_loss: 8.4997e-04 - val_mae: 0.0240

Epoch 00047: val_mae improved from 0.02411 to 0.02399, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 48/1000
132/132 - 3s - loss: 1.6730e-04 - mae: 0.0081 - val_loss: 8.3729e-04 - val_mae: 0.0238

Epoch 00048: val_mae improved from 0.02399 to 0.02377, saving model to ../../saved_models/water_all_95_cp.h5
Epoch 49/100

132/132 - 3s - loss: 1.2232e-04 - mae: 0.0068 - val_loss: 8.3033e-04 - val_mae: 0.0239

Epoch 00095: val_mae did not improve from 0.02297
Epoch 96/1000
132/132 - 3s - loss: 1.2161e-04 - mae: 0.0068 - val_loss: 8.5582e-04 - val_mae: 0.0244

Epoch 00096: val_mae did not improve from 0.02297
Epoch 97/1000
132/132 - 3s - loss: 1.2149e-04 - mae: 0.0068 - val_loss: 8.3356e-04 - val_mae: 0.0241

Epoch 00097: val_mae did not improve from 0.02297
Epoch 98/1000
132/132 - 3s - loss: 1.2075e-04 - mae: 0.0068 - val_loss: 7.9846e-04 - val_mae: 0.0235

Epoch 00098: val_mae did not improve from 0.02297
Epoch 99/1000
132/132 - 3s - loss: 1.2033e-04 - mae: 0.0067 - val_loss: 9.0002e-04 - val_mae: 0.0250

Epoch 00099: val_mae did not improve from 0.02297
Epoch 100/1000
132/132 - 3s - loss: 1.1921e-04 - mae: 0.0067 - val_loss: 8.7309e-04 - val_mae: 0.0246

Epoch 00100: val_mae did not improve from 0.02297
Epoch 101/1000
132/132 - 3s - loss: 1.1909e-04 - mae: 0.0067 - val_loss: 8.5745e-04 - val_mae: 0.0244

132/132 - 3s - loss: 1.0040e-04 - mae: 0.0061 - val_loss: 8.0550e-04 - val_mae: 0.0235

Epoch 00148: val_mae did not improve from 0.02240
Epoch 149/1000
132/132 - 3s - loss: 1.0043e-04 - mae: 0.0061 - val_loss: 7.4808e-04 - val_mae: 0.0226

Epoch 00149: val_mae did not improve from 0.02240
Epoch 150/1000
132/132 - 3s - loss: 1.0080e-04 - mae: 0.0061 - val_loss: 7.6337e-04 - val_mae: 0.0227

Epoch 00150: val_mae did not improve from 0.02240
Epoch 151/1000
132/132 - 3s - loss: 9.9496e-05 - mae: 0.0060 - val_loss: 8.2709e-04 - val_mae: 0.0237

Epoch 00151: val_mae did not improve from 0.02240
Epoch 152/1000
132/132 - 3s - loss: 1.0003e-04 - mae: 0.0061 - val_loss: 7.6426e-04 - val_mae: 0.0227

Epoch 00152: val_mae did not improve from 0.02240
Epoch 153/1000
132/132 - 3s - loss: 1.0099e-04 - mae: 0.0061 - val_loss: 7.4482e-04 - val_mae: 0.0225

Epoch 00153: val_mae did not improve from 0.02240
Epoch 154/1000
132/132 - 3s - loss: 9.9484e-05 - mae: 0.0060 - val_loss: 7.5661e-04 - val_mae: 0.

Epoch 200/1000
132/132 - 3s - loss: 9.1001e-05 - mae: 0.0057 - val_loss: 7.2379e-04 - val_mae: 0.0218

Epoch 00200: val_mae did not improve from 0.02093
Epoch 201/1000
132/132 - 3s - loss: 9.0751e-05 - mae: 0.0057 - val_loss: 6.7481e-04 - val_mae: 0.0210

Epoch 00201: val_mae did not improve from 0.02093
Epoch 202/1000
132/132 - 3s - loss: 9.1728e-05 - mae: 0.0057 - val_loss: 7.9478e-04 - val_mae: 0.0228

Epoch 00202: val_mae did not improve from 0.02093
Epoch 203/1000
132/132 - 3s - loss: 9.1241e-05 - mae: 0.0057 - val_loss: 7.5011e-04 - val_mae: 0.0221

Epoch 00203: val_mae did not improve from 0.02093
Epoch 204/1000
132/132 - 3s - loss: 9.0478e-05 - mae: 0.0057 - val_loss: 7.7118e-04 - val_mae: 0.0226

Epoch 00204: val_mae did not improve from 0.02093
Epoch 205/1000
132/132 - 3s - loss: 9.0790e-05 - mae: 0.0057 - val_loss: 6.9388e-04 - val_mae: 0.0213

Epoch 00205: val_mae did not improve from 0.02093
Epoch 206/1000
132/132 - 3s - loss: 9.0685e-05 - mae: 0.0057 - val_loss: 7.2589e-0

...


Epoch 00654: val_mae did not improve from 0.01660
Epoch 655/1000
132/132 - 3s - loss: 6.4883e-05 - mae: 0.0047 - val_loss: 5.3431e-04 - val_mae: 0.0177

Epoch 00655: val_mae did not improve from 0.01660
Epoch 656/1000
132/132 - 3s - loss: 6.4904e-05 - mae: 0.0047 - val_loss: 4.9883e-04 - val_mae: 0.0170

Epoch 00656: val_mae did not improve from 0.01660
Epoch 657/1000
132/132 - 4s - loss: 6.5041e-05 - mae: 0.0047 - val_loss: 5.1276e-04 - val_mae: 0.0171

Epoch 00657: val_mae did not improve from 0.01660
Epoch 658/1000
132/132 - 3s - loss: 6.5010e-05 - mae: 0.0047 - val_loss: 5.2746e-04 - val_mae: 0.0176

Epoch 00658: val_mae did not improve from 0.01660
Epoch 659/1000
132/132 - 3s - loss: 6.4745e-05 - mae: 0.0047 - val_loss: 5.0300e-04 - val_mae: 0.0171

Epoch 00659: val_mae did not improve from 0.01660
Epoch 660/1000
132/132 - 3s - loss: 6.4899e-05 - mae: 0.0047 - val_loss: 5.5619e-04 - val_mae: 0.0182

Epoch 00660: val_mae did not improve from 0.01660
Epoch 661/1000
132/132 - 3s - l

132/132 - 3s - loss: 6.3787e-05 - mae: 0.0046 - val_loss: 5.3896e-04 - val_mae: 0.0177

Epoch 00708: val_mae did not improve from 0.01637
Epoch 709/1000
132/132 - 3s - loss: 6.3814e-05 - mae: 0.0046 - val_loss: 5.6716e-04 - val_mae: 0.0183

Epoch 00709: val_mae did not improve from 0.01637
Epoch 710/1000
132/132 - 3s - loss: 6.4264e-05 - mae: 0.0047 - val_loss: 5.4581e-04 - val_mae: 0.0178

Epoch 00710: val_mae did not improve from 0.01637
Epoch 711/1000
132/132 - 3s - loss: 6.3913e-05 - mae: 0.0047 - val_loss: 5.3027e-04 - val_mae: 0.0176

Epoch 00711: val_mae did not improve from 0.01637
Epoch 712/1000
132/132 - 3s - loss: 6.3876e-05 - mae: 0.0046 - val_loss: 5.4682e-04 - val_mae: 0.0179

Epoch 00712: val_mae did not improve from 0.01637
Epoch 713/1000
132/132 - 3s - loss: 6.3833e-05 - mae: 0.0046 - val_loss: 5.4305e-04 - val_mae: 0.0177

Epoch 00713: val_mae did not improve from 0.01637
Epoch 714/1000
132/132 - 3s - loss: 6.3512e-05 - mae: 0.0046 - val_loss: 5.0527e-04 - val_mae: 0.


Epoch 00761: val_mae did not improve from 0.01637
Epoch 762/1000
132/132 - 3s - loss: 6.2961e-05 - mae: 0.0046 - val_loss: 5.5676e-04 - val_mae: 0.0180

Epoch 00762: val_mae did not improve from 0.01637
Epoch 763/1000
132/132 - 3s - loss: 6.3153e-05 - mae: 0.0046 - val_loss: 5.3629e-04 - val_mae: 0.0177

Epoch 00763: val_mae did not improve from 0.01637
Epoch 764/1000
132/132 - 3s - loss: 6.2667e-05 - mae: 0.0046 - val_loss: 5.5186e-04 - val_mae: 0.0178

Epoch 00764: val_mae did not improve from 0.01637
Epoch 765/1000
132/132 - 3s - loss: 6.2794e-05 - mae: 0.0046 - val_loss: 5.0326e-04 - val_mae: 0.0170

Epoch 00765: val_mae did not improve from 0.01637
Epoch 766/1000
132/132 - 3s - loss: 6.2830e-05 - mae: 0.0046 - val_loss: 5.5935e-04 - val_mae: 0.0181

Epoch 00766: val_mae did not improve from 0.01637
Epoch 767/1000
132/132 - 3s - loss: 6.2488e-05 - mae: 0.0046 - val_loss: 5.2238e-04 - val_mae: 0.0177

Epoch 00767: val_mae did not improve from 0.01637
Epoch 768/1000
132/132 - 3s - l

132/132 - 3s - loss: 6.2207e-05 - mae: 0.0046 - val_loss: 5.1401e-04 - val_mae: 0.0174

Epoch 00815: val_mae did not improve from 0.01622
Epoch 816/1000
132/132 - 3s - loss: 6.1839e-05 - mae: 0.0046 - val_loss: 5.2108e-04 - val_mae: 0.0172

Epoch 00816: val_mae did not improve from 0.01622
Epoch 817/1000
132/132 - 3s - loss: 6.2193e-05 - mae: 0.0046 - val_loss: 5.8238e-04 - val_mae: 0.0186

Epoch 00817: val_mae did not improve from 0.01622
Epoch 818/1000
132/132 - 3s - loss: 6.1709e-05 - mae: 0.0046 - val_loss: 5.1076e-04 - val_mae: 0.0172

Epoch 00818: val_mae did not improve from 0.01622
Epoch 819/1000
132/132 - 3s - loss: 6.1815e-05 - mae: 0.0046 - val_loss: 4.8839e-04 - val_mae: 0.0167

Epoch 00819: val_mae did not improve from 0.01622
Epoch 820/1000
132/132 - 3s - loss: 6.1839e-05 - mae: 0.0046 - val_loss: 5.1258e-04 - val_mae: 0.0174

Epoch 00820: val_mae did not improve from 0.01622
Epoch 821/1000
132/132 - 3s - loss: 6.1666e-05 - mae: 0.0046 - val_loss: 4.9584e-04 - val_mae: 0.


Epoch 00868: val_mae did not improve from 0.01622
Epoch 869/1000
132/132 - 3s - loss: 6.1033e-05 - mae: 0.0045 - val_loss: 5.4401e-04 - val_mae: 0.0179

Epoch 00869: val_mae did not improve from 0.01622
Epoch 870/1000
132/132 - 3s - loss: 6.1431e-05 - mae: 0.0046 - val_loss: 5.2045e-04 - val_mae: 0.0175

Epoch 00870: val_mae did not improve from 0.01622
Epoch 871/1000
132/132 - 3s - loss: 6.0926e-05 - mae: 0.0045 - val_loss: 5.3401e-04 - val_mae: 0.0176

Epoch 00871: val_mae did not improve from 0.01622
Epoch 872/1000
132/132 - 3s - loss: 6.1048e-05 - mae: 0.0045 - val_loss: 5.1717e-04 - val_mae: 0.0175

Epoch 00872: val_mae did not improve from 0.01622
Epoch 873/1000
132/132 - 3s - loss: 6.1372e-05 - mae: 0.0046 - val_loss: 5.3938e-04 - val_mae: 0.0177

Epoch 00873: val_mae did not improve from 0.01622
Epoch 00873: early stopping
